In [3]:
import yfinance as yf
import pandas as pd
import time

n = 50 
excel_file = '股票代码.xlsx'
# ===========================================

print(f"正在读取前 {n} 个股票代码...")

try:
    df = pd.read_excel(excel_file)
    
    raw_tickers = df.iloc[:n, 0].astype(str).tolist()
    
    # 清理股票代码：去掉.O .N 等后缀
    ticker_list = []
    for ticker in raw_tickers:
        if ticker and ticker != 'nan':
            # 去掉常见后缀
            cleaned = ticker.replace('.O', '').replace('.N', '').replace('.A', '').strip().upper()
            if len(cleaned) > 0 and any(c.isalpha() for c in cleaned):
                ticker_list.append(cleaned)
    
    print(f"成功获取 {len(ticker_list)} 个股票代码")
    
except Exception as e:
    print(f"读取文件失败: {e}")
    exit()

for i, ticker_symbol in enumerate(ticker_list, 1):
    
    print(f"[{i}/{len(ticker_list)}] 正在处理 {ticker_symbol}...")
    
    start_date = '2010-01-01'
    end_date = '2025-07-23'
    
    ticker = yf.Ticker(ticker_symbol)

    data = yf.download(
        ticker_symbol,
        start=start_date,
        end=end_date,
        auto_adjust=True,
        progress=False
    )
    
    daily_data_success = False
    if not data.empty:
        data = data.reset_index()
        data['Date'] = data['Date'].dt.date
        data.columns = ['日期', '收盘价', '最高价', '最低价', '开盘价', '交易量']
        data = data[['日期', '开盘价', '最高价', '最低价', '收盘价', '交易量']]
        
        # 格式化数值
        for col in ['开盘价', '最高价', '最低价', '收盘价']:
            data[col] = data[col].round(2)
        data['交易量'] = data['交易量'].astype('int64')
        
        daily_data_success = True
        print(f"  日线数据: {len(data)} 个交易日")
    
    company_info_success = False
    try:
        info = ticker.info
        
        all_company_info = {
            "公司名称": info.get('longName', 'N/A'),
            "股票代码": info.get('symbol', ticker_symbol),
            "所属行业": info.get('sector', 'N/A'),
            "业务类型": info.get('industry', 'N/A'),
            "国家": info.get('country', 'N/A'),
            "城市": info.get('city', 'N/A'),
            "公司网站": info.get('website', 'N/A'),
            "员工数量": f"{info.get('fullTimeEmployees', 0):,}" if info.get('fullTimeEmployees') else 'N/A',
            "交易所": info.get('exchange', 'N/A'),
            "货币": info.get('currency', 'N/A'),
            "当前价格": f"${info.get('currentPrice', 'N/A')}" if info.get('currentPrice') else 'N/A',
            "前收盘价": f"${info.get('previousClose', 'N/A')}" if info.get('previousClose') else 'N/A',
            "52周最高": f"${info.get('fiftyTwoWeekHigh', 'N/A')}" if info.get('fiftyTwoWeekHigh') else 'N/A',
            "52周最低": f"${info.get('fiftyTwoWeekLow', 'N/A')}" if info.get('fiftyTwoWeekLow') else 'N/A',
            "当前市值(美元)": f"${info.get('marketCap', 0):,}" if info.get('marketCap') else 'N/A',
            "当前市值(亿美元)": f"{info.get('marketCap', 0)/1e8:.2f}" if info.get('marketCap') else 'N/A',
            "市盈率(PE)": f"{info.get('trailingPE', 'N/A'):.2f}" if info.get('trailingPE') else 'N/A',
            "市净率(PB)": f"{info.get('priceToBook', 'N/A'):.2f}" if info.get('priceToBook') else 'N/A',
            "股息收益率": f"{info.get('dividendYield', 0)*100:.2f}%" if info.get('dividendYield') else 'N/A',
            "每股收益(EPS)": f"${info.get('trailingEps', 'N/A')}" if info.get('trailingEps') else 'N/A',
            "流通股本": f"{info.get('sharesOutstanding', 0):,}" if info.get('sharesOutstanding') else 'N/A',
            "企业价值": f"${info.get('enterpriseValue', 0):,}" if info.get('enterpriseValue') else 'N/A',
            "总收入": f"${info.get('totalRevenue', 0):,}" if info.get('totalRevenue') else 'N/A',
            "总债务": f"${info.get('totalDebt', 0):,}" if info.get('totalDebt') else 'N/A',
            "总现金": f"${info.get('totalCash', 0):,}" if info.get('totalCash') else 'N/A'
        }
        
        company_df = pd.DataFrame.from_dict(all_company_info, orient='index', columns=['数值'])
        company_df.index.name = '指标'
        company_info_success = True
        print(f"  公司信息: {len(all_company_info)} 项指标")
    
    except Exception as e:
        print(f"  公司信息获取失败: {e}")

    try:
        if daily_data_success:
            actual_end_date = data['日期'].max().strftime('%Y-%m-%d')
        else:
            actual_end_date = pd.Timestamp.now().strftime('%Y-%m-%d')
            
        filename = f"{ticker_symbol}_完整数据_{actual_end_date}.xlsx"
    
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            if daily_data_success:
                data.to_excel(writer, sheet_name='日线数据', index=False)
            if company_info_success:
                company_df.to_excel(writer, sheet_name='公司信息')
        
        print(f"  数据已保存: {filename}")
    
    except Exception as e:
        print(f"  保存失败: {e}")
    
    # 暂停1秒避免API限制
    time.sleep(1)

print(f"\n数据收集完成！共处理 {len(ticker_list)} 只股票")

正在读取前 50 个股票代码...
成功获取 50 个股票代码
[1/50] 正在处理 NVDA...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: NVDA_完整数据_2025-07-22.xlsx
[2/50] 正在处理 MSFT...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: MSFT_完整数据_2025-07-22.xlsx
[3/50] 正在处理 AAPL...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: AAPL_完整数据_2025-07-22.xlsx
[4/50] 正在处理 AMZN...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: AMZN_完整数据_2025-07-22.xlsx
[5/50] 正在处理 GOOG...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: GOOG_完整数据_2025-07-22.xlsx
[6/50] 正在处理 GOOGL...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: GOOGL_完整数据_2025-07-22.xlsx
[7/50] 正在处理 META...
  日线数据: 3312 个交易日
  公司信息: 25 项指标
  数据已保存: META_完整数据_2025-07-22.xlsx
[8/50] 正在处理 AVGO...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: AVGO_完整数据_2025-07-22.xlsx
[9/50] 正在处理 TSM...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: TSM_完整数据_2025-07-22.xlsx
[10/50] 正在处理 TSLA...
  日线数据: 3789 个交易日
  公司信息: 25 项指标
  数据已保存: TSLA_完整数据_2025-07-22.xlsx
[11/50] 正在处理 BRK_B...


HTTP Error 404: 

1 Failed download:
['BRK_B']: YFTzMissingError('possibly delisted; no timezone found')
HTTP Error 404: 


  公司信息: 25 项指标
  数据已保存: BRK_B_完整数据_2025-07-25.xlsx
[12/50] 正在处理 BRK_A...


HTTP Error 404: 

1 Failed download:
['BRK_A']: YFTzMissingError('possibly delisted; no timezone found')
HTTP Error 404: 


  公司信息: 25 项指标
  数据已保存: BRK_A_完整数据_2025-07-25.xlsx
[13/50] 正在处理 JPM...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: JPM_完整数据_2025-07-22.xlsx
[14/50] 正在处理 WMT...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: WMT_完整数据_2025-07-22.xlsx
[15/50] 正在处理 LLY...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: LLY_完整数据_2025-07-22.xlsx
[16/50] 正在处理 ORCL...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: ORCL_完整数据_2025-07-22.xlsx
[17/50] 正在处理 V...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: V_完整数据_2025-07-22.xlsx
[18/50] 正在处理 MA...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: MA_完整数据_2025-07-22.xlsx
[19/50] 正在处理 NFLX...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: NFLX_完整数据_2025-07-22.xlsx
[20/50] 正在处理 XOM...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: XOM_完整数据_2025-07-22.xlsx
[21/50] 正在处理 COST...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: COST_完整数据_2025-07-22.xlsx
[22/50] 正在处理 JNJ...



1 Failed download:
['JNJ']: SSLError('Failed to perform, curl: (35) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


  公司信息: 25 项指标
  数据已保存: JNJ_完整数据_2025-07-25.xlsx
[23/50] 正在处理 PG...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: PG_完整数据_2025-07-22.xlsx
[24/50] 正在处理 HD...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: HD_完整数据_2025-07-22.xlsx
[25/50] 正在处理 SAP...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: SAP_完整数据_2025-07-22.xlsx
[26/50] 正在处理 BAC...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: BAC_完整数据_2025-07-22.xlsx
[27/50] 正在处理 PLTR...
  日线数据: 1207 个交易日
  公司信息: 25 项指标
  数据已保存: PLTR_完整数据_2025-07-22.xlsx
[28/50] 正在处理 ABBV...
  日线数据: 3157 个交易日
  公司信息: 25 项指标
  数据已保存: ABBV_完整数据_2025-07-22.xlsx
[29/50] 正在处理 KO...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: KO_完整数据_2025-07-22.xlsx
[30/50] 正在处理 BABA...
  日线数据: 2725 个交易日
  公司信息: 25 项指标
  数据已保存: BABA_完整数据_2025-07-22.xlsx
[31/50] 正在处理 ASML...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: ASML_完整数据_2025-07-22.xlsx
[32/50] 正在处理 GE...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: GE_完整数据_2025-07-22.xlsx
[33/50] 正在处理 CSCO...
  日线数据: 3911 个交易日
  公司信息: 25 项指标
  数据已保存: CSCO_完整数据_2025-07-